In [ ]:
import requests
import io 
import pandas as pd 

In [ ]:
fips_url = 'https://www.bls.gov/cew/classifications/areas/area-titles-csv.csv'
estb_url = 'http://www.bls.gov/cew/data/api/2021/3/industry/6221.csv'
ownc_url = 'https://www.bls.gov/cew/classifications/ownerships/sic-ownership-titles-csv.csv'

In [ ]:
def get_dataframe(url: str)->pd.DataFrame:
    """
    fetch data from url convert that data into dataframe
    
    Args:
    ----
    String : url 
             url of csv file 
    
    Returns
    ------
    Dataframe

    Example
    --------
    >>> ownc_url = 'https://www.bls.gov/cew/classifications/ownerships/sic-ownership-titles-csv.csv'
    >>> df = get_dataframe(ownc_url)
    >>> df
    own_code	ownership_title
    0	0	Total Covered
    1	5	Private
    2	4	International Government
    3	3	Local Government
    4	2	State Government
    5	1	Federal Government
    """
    data = requests.get(url)
    content = data.content
    return pd.read_csv(io.StringIO(content.decode('utf-8')))

In [ ]:
estb_df = get_dataframe(estb_url)
# fips_df = get_dataframe(fips_url)
# ownc_df = get_dataframe(ownc_url)
ownc_df.rename(columns={'ownership_code':'own_code'}, inplace=True)
fips_df['area_fips'] = fips_df['area_fips'].apply(lambda x : x if len(x)==5 else '0'+str(x) )
estb_df = estb_df[['area_fips', 'own_code', 'qtrly_estabs','year','qtr']]
df = estb_df.merge(fips_df, on=['area_fips'], how='left')
df = df.merge(ownc_df, on=['own_code'], how='left')
df.head()

In [ ]:
ownc_df

# Combine files from a folder 

In [ ]:
from pathlib import Path
import pandas as pd

def combine_files_into_df(input_path_object, file_pattern):
#     joined_list = Path().absolute().joinpath('orders').glob('*.csv')
    joined_list = input_path_object.glob(file_pattern)
    df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
    return df

#Combining multiple quarters files
df_2018 = combine_files_into_df(Path().absolute().joinpath('2018'),'*.csv')
df_2019 = combine_files_into_df(Path().absolute().joinpath('2019'),'*.csv')
df_2020 = combine_files_into_df(Path().absolute().joinpath('2020'),'*.csv')
df_2021 = combine_files_into_df(Path().absolute().joinpath('2021'),'*.csv')

#Combinig multiple years dataframe
df_master = pd.concat([df_2018, df_2019, df_2020, df_2021], axis = 'rows')
df_master.shape

#exporting the master dataframe of all the years
df_master.to_csv('output.csv')